In [1]:
import pandas as pd

import optuna
from catboost import CatBoostRanker, Pool
from scipy.stats import spearmanr, kendalltau

from tools import train_valid_test_split, calculate_ndcg_per_query

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
pca_20_features = "data/pca_20_features.csv"

In [3]:
data_df = pd.read_csv(pca_20_features)

## Test/Train Split

In [4]:
train_data, valid_data, test_data = train_valid_test_split(data_df, test_size=0.10, valid_size=0.10)

In [5]:
to_drop = ["rank", "query_id"]

X_train = train_data.drop(to_drop, axis=1)
y_train = train_data["rank"]

X_valid = valid_data.drop(to_drop, axis=1)
y_valid = valid_data["rank"]

X_test = test_data.drop(to_drop, axis=1)
y_test = test_data[["query_id", "rank"]]  # query_id for ndcg score calculation

In [6]:
X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape

((190164, 20), (22534, 20), (22560, 20), (190164,), (22534,), (22560, 2))

In [7]:
train_pool = Pool(data=X_train, label=y_train, group_id=train_data["query_id"])
valid_pool = Pool(data=X_valid, label=y_valid, group_id=valid_data["query_id"])
test_pool = Pool(data=X_test, label=y_test, group_id=test_data["query_id"])

## Catboost Baseline Model

In [8]:
def objective(trial):
    param = {
        "iterations": trial.suggest_int("iterations", 300, 800),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 4, 8),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 5, 15)
    }
    
    model = CatBoostRanker(loss_function="YetiRank", **param)
    model.fit(train_pool, eval_set=valid_pool, verbose=False)
    
    predictions = model.predict(test_pool)
    mean_ndcg = calculate_ndcg_per_query(y_test, predictions, k=30, pct_k=0.2)
    return mean_ndcg

In [9]:
is_study = False

if is_study:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    
    print("Best trial:")
    best_trial = study.best_trial
    
    print(f"NDCG: {best_trial.value}")
    print("Best hyperparameters:")
    print(best_trial.params)

## Рассчёт Метрик
Возьмём модель с лучшими параметрами и посчитаем пару метрик.

In [10]:
best_args = {"iterations": 480, "learning_rate": 0.09, "depth": 5, "l2_leaf_reg": 10}
model = CatBoostRanker(loss_function="YetiRank", **best_args)

In [11]:
model.fit(train_pool, eval_set=valid_pool, verbose=False)

In [12]:
predict = model.predict(test_pool)

In [13]:
spearman_corr = spearmanr(y_test["rank"].values, predict).correlation
kendall_corr = kendalltau(y_test["rank"].values, predict).correlation
ndcg_1 = calculate_ndcg_per_query(y_test, predict, k=1)
ndcg_5 = calculate_ndcg_per_query(y_test, predict, k=5)
ndcg_30 = calculate_ndcg_per_query(y_test, predict, k=30)
ndcg_pct_20 = calculate_ndcg_per_query(y_test, predict, k=30, pct_k=0.2)
ndcg_pct_50 = calculate_ndcg_per_query(y_test, predict, k=30, pct_k=0.5)
ndcg_pct_100 = calculate_ndcg_per_query(y_test, predict, k=30, pct_k=1)

In [14]:
print(f"Spearman correlation: {spearman_corr}")
print(f"Kendall correlation: {kendall_corr}")
print(f"NDCG@1: {ndcg_1}")
print(f"NDCG@5: {ndcg_5}")
print(f"NDCG@30: {ndcg_30}")
print(f"NDCG pct=0.2: {ndcg_pct_20}")
print(f"NDCG pct=0.5: {ndcg_pct_50}")
print(f"NDCG pct=1: {ndcg_pct_100}")

Spearman correlation: 0.36764885237647704
Kendall correlation: 0.28842293201453717
NDCG@1: 0.46608040201005024
NDCG@5: 0.47767284239885016
NDCG@30: 0.5385616866419879
NDCG pct=0.2: 0.5396160280607956
NDCG pct=0.5: 0.5957256525216853
NDCG pct=1: 0.7426640507382953


Корреляции положительные, однако относительно небольшие.
Из NDCG метрик можно сделать вывод, что модель не очень хорошо ранжирует документы. С увеличением числа документов - растёт метрика.
Думаю, что дисбаланс классов достаточно сильно влияет на результаты.

Что можно было бы еще сделать:
1. Посчитать метрики для моделей классификации (f1, precision, recall, roc-auc), чтобы отдельно оценить точность определения конкретного ранга.
2. Учесть дисбаланс классов: попробовать oversampling/undersampling, или применить другой алгоритм машинного обучения, который поддерживает взвешивание классов.